In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [46]:
segments = pd.read_csv("../dataset/strava-segments-italy-FINAL.csv")

In [47]:
segments[["s_lat", "s_lng"]] = segments["start_latlng"].apply(lambda x: pd.Series(np.array(eval(x))))

In [48]:
segments[["e_lat", "e_lng"]] = segments["end_latlng"].apply(lambda x: pd.Series(np.array(eval(x))))

In [49]:
segments = segments.drop(columns=["start_latlng", "end_latlng"])

In [53]:
segments.drop("Unnamed: 0", axis=1, inplace=True)

In [54]:
segments.to_csv(path_or_buf="../dataset/segments-italy-full-preprocessed.csv", index=True)

In [5]:
activities = pd.read_csv("../dataset/strava-activities-italy-full-FINAL.csv")

In [6]:
activities.drop("Unnamed: 0", axis=1, inplace=True)
activities.drop("Unnamed: 0.1", axis=1, inplace=True)

In [7]:
activities = activities.dropna()

In [8]:
activities["segment"] = activities["segment"].astype(int, copy=True)

In [9]:
activities[activities["start_date"].isnull()]["start_date"]

Series([], Name: start_date, dtype: object)

In [10]:
indexes_with_wrong_dates = activities[activities["start_date"].str.match(r"[234][1-9]\d\d-*")].index

In [11]:
activities.drop(indexes_with_wrong_dates, inplace=True)

In [12]:
indexes_with_wrong_dates = activities[activities["start_date"].str.match(r"[4]\d\d\d-*")].index

In [13]:
activities.drop(indexes_with_wrong_dates, inplace=True)

In [15]:
activities["start_date"] = pd.to_datetime(activities["start_date"])

In [17]:
 activities["Year"] = activities["start_date"].dt.year

In [19]:
 activities["Month"] = activities["start_date"].dt.month

In [20]:
 activities["Day"] = activities["start_date"].dt.day

In [25]:
 activities["Dow"] = activities["start_date"].dt.dayofweek

In [31]:
activities_parsed = pd.merge(activities, segments, left_on="segment", right_on="id").drop(["id", "points", "start_latlng", "end_latlng"], axis=1)

In [32]:
activities_parsed.head()

,athlete_name,elapsed_time,moving_time,rank,start_date,start_date_local,segment,Year,Month,Day,...,climb_category_desc,distance,elev_difference,name,resource_state,starred,s_lat,s_lng,e_lat,e_lng
0,Dan B.,275,275,1,2013-04-05 08:49:39,2013-04-05T10:49:39Z,2390852,2013,4,5,...,4,2069.3,107.4,San Martin,2,False,35.940484,14.376255,35.927646,14.377224
1,Etienne B.,276,276,2,2013-04-05 08:49:41,2013-04-05T10:49:41Z,2390852,2013,4,5,...,4,2069.3,107.4,San Martin,2,False,35.940484,14.376255,35.927646,14.377224
2,James M.,287,287,3,2014-04-04 08:28:23,2014-04-04T10:28:23Z,2390852,2014,4,4,...,4,2069.3,107.4,San Martin,2,False,35.940484,14.376255,35.927646,14.377224
3,Tom B.,288,288,4,2014-04-04 08:28:26,2014-04-04T10:28:26Z,2390852,2014,4,4,...,4,2069.3,107.4,San Martin,2,False,35.940484,14.376255,35.927646,14.377224
4,Corrado C.,290,290,5,2014-04-04 08:28:25,2014-04-04T10:28:25Z,2390852,2014,4,4,...,4,2069.3,107.4,San Martin,2,False,35.940484,14.376255,35.927646,14.377224


In [33]:
activities_parsed.to_csv(path_or_buf="../dataset/activities-italy-full-preprocessed.csv", index=False)